In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import cv2
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import csv

In [40]:
import pandas as pd

# Leer el archivo XLSX
df = pd.read_excel('INE 2022.xlsx', sheet_name=0, header=None, names=['PROVINCIA', 'MUNICIPIO', 'POB22'])

In [42]:
# Eliminar las filas 0 y 1 del DataFrame
df = df.drop([0, 1])

# Crear una lista con la columna de municipios actualizada
municipios = df['MUNICIPIO'].tolist()

In [9]:
# Diccionario para almacenar la información
datos_electorales = {}

In [10]:
# Función para esperar en segundos
def esperar(segundos):
    time.sleep(segundos)

In [35]:

# Inicializar el driver de Selenium con Chrome
driver = webdriver.Chrome()

# Abrir la página web
driver.get("https://resultados.locales2023.es/")

# Buscar y escribir en el campo de búsqueda por cada ciudad
for ciudad in municipios:
    search_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='q']")))
    search_box.click()
    search_box.clear()
    search_box.send_keys(ciudad)
    search_box.send_keys(Keys.RETURN)

    # Esperar a que se cargue la página con los resultados
    time.sleep(20)  # Ajusta el tiempo de espera según sea necesario

    # Obtener la información de los partidos y concejales
    partido_elementos = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.min-w-[10px] max-w-[95px] truncate whitespace-nowrap text-left.text-sm.text-primary-800")))
    concejales_elementos = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "span.text-base.font-bold.text-primary-800")))

    partidos = [partido.text for partido in partido_elementos]
    concejales = [concejal.text for concejal in concejales_elementos]

    # Almacenar la información en el diccionario
    datos_electorales[ciudad] = {
        "partidos": partidos,
        "concejales": concejales
    }

    # Esperar 3 minutos antes de la siguiente búsqueda
    time.sleep(20)  # Tiempo de espera de 3 minutos (180 segundos)

# Cerrar el navegador
driver.quit()

# Imprimir los datos electorales
for ciudad, datos in datos_electorales.items():
    print("Ciudad:", ciudad)
    print("Partidos:", datos["partidos"])
    print("Concejales:", datos["concejales"])
    print()

StaleElementReferenceException: Message: A node reference could not be resolved: Node with identifier 'node-6058C56E-71B1-4197-8D0E-352A8A728BA3' was not found


In [ ]:
# Convertir el diccionario en un DataFrame
df_datos_electorales = pd.DataFrame.from_dict(datos_electorales, orient='index')

# Imprimir el DataFrame
print(df_datos_electorales)